In [ ]:
!pip install ultralytics --quiet

from ultralytics import YOLO
import pandas as pd
import os
import shutil

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)

folder_path = '/content/gdrive/My Drive/DadosTcc2'

try:
    for item in os.listdir(folder_path):
        print(item)
except FileNotFoundError:
    print(f"Error: The folder '{folder_path}' was not found.")
except Exception as e:
    print(f"An error occurred: {e}")

In [ ]:
base_path = folder_path
dataset_path = os.path.join(base_path, "dataset")
labels_path = os.path.join(base_path, "labels2.csv")
yolo_base = os.path.join(base_path, "yolo_dataset")

os.makedirs(yolo_base, exist_ok=True)

df = pd.read_csv(labels_path)

print("Colunas encontradas:", df.columns.tolist())

def prepare_yolo_folder(label_name):
    label_dir = os.path.join(yolo_base, label_name)
    for split in ["train", "val"]:
        for cls in ["0", "1"]:
            os.makedirs(os.path.join(label_dir, split, cls), exist_ok=True)
    return label_dir

from sklearn.model_selection import train_test_split

train_df, val_df = train_test_split(df, test_size=0.2, random_state=42)

def create_yolo_dataset(df_split, label, split_name):
    for _, row in df_split.iterrows():
        image_filename = row["image"] + ".jpg"
        src = os.path.join(dataset_path, image_filename)
        dst_dir = os.path.join(yolo_base, label, split_name, str(row[label]))
        if os.path.exists(src):
            shutil.copy(src, dst_dir)
        else:
            print(f"Arquivo de imagem não encontrado em {src}")

for label in ["contrast", "proximity", "alignment"]:
    label_dir = prepare_yolo_folder(label)
    create_yolo_dataset(train_df, label, "train")
    create_yolo_dataset(val_df, label, "val")

def train_yolo_classifier(label):
    data_path = os.path.join(yolo_base, label)
    model_name = f"{label}_model"
    print(f"Treinando modelo para '{label}'...")
    !yolo classify train model=yolov8n-cls.pt data="{data_path}" epochs=20 imgsz=224 name={model_name}

for label in ["contrast", "proximity", "alignment"]:
    train_yolo_classifier(label)

In [ ]:
def validate_yolo_classifier(label):
    model_path = f"/content/runs/classify/{label}_model2/weights/best.pt"
    data_path = os.path.join(yolo_base, label)
    print(f"Validação do modelo '{label}'...")
    !yolo classify val model="{model_path}" data="{data_path}"

for label in ["contrast", "proximity", "alignment"]:
    validate_yolo_classifier(label)